In [10]:
import csv
import numpy as np
import pandas as pd
import scipy
import matplotlib.pyplot as plt
import time
import math

In [11]:
with open('data/train.csv') as datafile:
    filecontents = csv.reader(datafile, delimiter = ",")
    data = list(filecontents)



In [12]:
drivers = pd.DataFrame(data)
drivers.columns = drivers.iloc[0]
drivers = drivers.set_index(['id'])
drivers = drivers[1:]
drivers

,target,ps_ind_01,ps_ind_02_cat,ps_ind_03,ps_ind_04_cat,ps_ind_05_cat,ps_ind_06_bin,ps_ind_07_bin,ps_ind_08_bin,ps_ind_09_bin,...,ps_calc_11,ps_calc_12,ps_calc_13,ps_calc_14,ps_calc_15_bin,ps_calc_16_bin,ps_calc_17_bin,ps_calc_18_bin,ps_calc_19_bin,ps_calc_20_bin
id,,,,,,,,,,,,,,,,,,,,,
7,0,2,2,5,1,0,0,1,0,0,...,9,1,5,8,0,1,1,0,0,1
9,0,1,1,7,0,0,0,0,1,0,...,3,1,1,9,0,1,1,0,1,0
13,0,5,4,9,1,0,0,0,1,0,...,4,2,7,7,0,1,1,0,1,0
16,0,0,1,2,0,0,1,0,0,0,...,2,2,4,9,0,0,0,0,0,0
17,0,0,2,0,1,0,1,0,0,0,...,3,1,1,3,0,0,0,1,1,0
19,0,5,1,4,0,0,0,0,0,1,...,4,2,0,9,0,1,0,1,1,1
20,0,2,1,3,1,0,0,1,0,0,...,3,0,0,10,0,1,0,0,1,0
22,0,5,1,4,0,0,1,0,0,0,...,7,1,3,6,1,0,1,0,1,0
26,0,5,1,3,1,0,0,0,1,0,...,4,2,1,5,0,1,0,0,0,1


In [13]:
from sklearn.tree import DecisionTreeClassifier

msk = np.random.rand(len(drivers)) < 0.8 # vector the length of drivers that has random Trues or Falses, with 80% Trues
# create train and test sets
drivers_train = drivers[msk]
drivers_test = drivers[~msk]
#remove answers (all of this is from nicole's other project)
y_train = drivers_train['target']
y_test = drivers_test['target']
x_train = drivers_train.drop('target', axis = 1)
x_test = drivers_test.drop('target', axis = 1)

#create the tree
dt = DecisionTreeClassifier(min_samples_split=1600)
dt.fit(x_train, y_train)

DecisionTreeClassifier(class_weight=None, criterion='gini', max_depth=None,
            max_features=None, max_leaf_nodes=None,
            min_impurity_decrease=0.0, min_impurity_split=None,
            min_samples_leaf=1, min_samples_split=1600,
            min_weight_fraction_leaf=0.0, presort=False, random_state=None,
            splitter='best')

In [14]:
claims_pred = list(dt.predict(x_test))

from sklearn.metrics import confusion_matrix
matrix = confusion_matrix(y_test, claims_pred)
matrix

array([[114730,     35],
       [  4300,      3]], dtype=int64)

In [32]:
#sort out the bad ad good drivers

bad_drivers = drivers[drivers['target'] == '1'] #How long did it take Maggie to realize this was the string 1,
good_drivers = drivers[drivers['target'] == '0'] #rather than the integer?

#take a short list of good drivers, approximately equal to the number of bad drivers in the training set
#I got .008 because there are about .01 as many bad drivers as good, * .8 for the training set
msk = np.random.rand(len(good_drivers)) <0.008 
short_good_drivers = good_drivers[msk]
long_good_drivers = good_drivers[~msk]

bad_msk = np.random.rand(len(bad_drivers)) < 0.8
good_msk = np.random.rand(len(long_good_drivers)) < 0.2 #trying to get the right proportion of good drivers to bad in the test set.

bad_train = bad_drivers[bad_msk]
bad_test = bad_drivers[~bad_msk]
good_train = short_good_drivers
good_test = long_good_drivers [good_msk]

#cmerge the bad and good drivers back into the same dataframes
trains = [bad_train, good_train]
train_frame = pd.concat(trains)

tests = [bad_test, good_test]
test_frame = pd.concat(tests)



In [33]:
#remove answers (all of this is from nicole's other project)
y_train = train_frame['target']
y_test = test_frame['target']
x_train = train_frame.drop('target', axis = 1)
x_test = test_frame.drop('target', axis = 1)

#create the tree
dt = DecisionTreeClassifier(min_samples_split=1600)
dt.fit(x_train, y_train)

#make some predictions!
claims_pred = list(dt.predict(x_test))

#check the predictions
from sklearn.metrics import confusion_matrix
matrix = confusion_matrix(y_test, claims_pred)
matrix

array([[   326, 112527],
       [    14,   4265]], dtype=int64)

In [27]:
drivers['ps_ind_01'] == '2'



id
7           True
9          False
13         False
16         False
17         False
19         False
20          True
22         False
26         False
28         False
34         False
35          True
36          True
43         False
46         False
48         False
50         False
58         False
61         False
64         False
65         False
66         False
72         False
74          True
77         False
78         False
79         False
80         False
84         False
85         False
           ...  
1487945    False
1487951    False
1487952    False
1487954    False
1487957    False
1487958    False
1487962     True
1487963    False
1487964    False
1487968    False
1487973    False
1487975    False
1487976    False
1487980    False
1487983    False
1487988    False
1487990    False
1487992    False
1487994    False
1487996    False
1488001    False
1488005    False
1488008    False
1488009    False
1488011    False
1488013    False
1488016    False
1488017    